In [1]:
from sqlalchemy import create_engine
from sqlalchemy import Integer, String, ForeignKey
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship
from typing import List, Optional
from sqlalchemy import insert
from sqlalchemy.orm import Session, aliased
from sqlalchemy import select, bindparam
from sqlalchemy import literal_column
from sqlalchemy import and_, or_, func
from sqlalchemy import asc, desc

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]

    addresses: Mapped[List["Address"]] = relationship(back_populates="user")

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name}, fullname={self.fullname})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))


    user: Mapped[List["User"]] = relationship(back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id}, user_id={self.user_id}, email_address={self.email_address})"


engine = create_engine("sqlite+pysqlite:///instances/user.db", echo=True)



In [2]:
Base.metadata.create_all(engine)

2023-12-10 10:35:35,184 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-10 10:35:35,191 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-12-10 10:35:35,192 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-10 10:35:35,196 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-12-10 10:35:35,199 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-10 10:35:35,201 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
insert_stmt = insert(User).values(
    [{"name": "sandy", "fullname": "Sandy Miles"},
     {"name": "tony", "fullname":"Tony Stark"},
     {"name": "thor", "fullname": "The Mighty Thor"}]
)

with Session(engine) as conn:
    result = conn.execute(insert_stmt)
    conn.commit()

2023-12-06 11:35:17,960 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-06 11:35:17,993 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?), (?, ?), (?, ?)
2023-12-06 11:35:18,001 INFO sqlalchemy.engine.Engine [no key 0.00929s] ('sandy', 'Sandy Miles', 'tony', 'Tony Stark', 'thor', 'The Mighty Thor')
2023-12-06 11:35:18,025 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
sub_query = select(User.id).where(User.name == bindparam("username")).scalar_subquery()

with Session(engine) as session:
    result = session.execute(insert(Address).values(user_id=sub_query),
                             [
                               {"username": "sandy", "email_address": "sandy@aol.com"},
                               {"username": "tony", "email_address": "tony.start@aol.com"},
                               {"username": "thor", "email_address": "baby.thor@aol.com"}
                             ])
    session.commit()



2023-12-06 11:51:04,287 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-06 11:51:04,308 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, (SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?))
2023-12-06 11:51:04,319 INFO sqlalchemy.engine.Engine [cached since 53.24s ago] [('sandy@aol.com', 'sandy'), ('tony.start@aol.com', 'tony'), ('baby.thor@aol.com', 'thor')]
2023-12-06 11:51:04,325 INFO sqlalchemy.engine.Engine COMMIT


In [2]:
sub_query = select(User.id).where(User.name == bindparam("username")).scalar_subquery()

with Session(engine) as session:
    result = session.execute(insert(Address).values(user_id=sub_query),
                             [
                               {"username": "sandy", "email_address": "sandy@example.com"},
                               {"username": "tony", "email_address": "tony.start@example.com"},
                               {"username": "thor", "email_address": "baby.thor@example.com"}
                             ])
    session.commit()



2023-12-09 10:25:15,734 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-09 10:25:15,871 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, (SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?))
2023-12-09 10:25:15,873 INFO sqlalchemy.engine.Engine [generated in 0.00192s] [('sandy@example.com', 'sandy'), ('tony.start@example.com', 'tony'), ('baby.thor@example.com', 'thor')]
2023-12-09 10:25:15,879 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
stmt = select(User).where(User.name == "sandy")
print(stmt)
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1
2023-12-10 10:36:11,656 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-10 10:36:11,662 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2023-12-10 10:36:11,663 INFO sqlalchemy.engine.Engine [generated in 0.00104s] ('sandy',)
(User(id=1, name=sandy, fullname=Sandy Miles),)
2023-12-10 10:36:11,670 INFO sqlalchemy.engine.Engine ROLLBACK


In [5]:
row = session.execute(select(User)).first()
print(row)

2023-12-07 10:35:36,817 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-07 10:35:36,827 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2023-12-07 10:35:36,833 INFO sqlalchemy.engine.Engine [generated in 0.00611s] ()
(User(id=1, name=sandy, fullname=Sandy Miles),)


In [6]:
row = session.scalars(select(User)).fetchall()
print(row)

2023-12-07 10:35:40,125 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2023-12-07 10:35:40,128 INFO sqlalchemy.engine.Engine [cached since 3.301s ago] ()
[User(id=1, name=sandy, fullname=Sandy Miles), User(id=2, name=tony, fullname=Tony Stark), User(id=3, name=thor, fullname=The Mighty Thor)]


In [8]:
row = session.execute(select(User.name, User.fullname)).first()
print(row)

2023-12-07 10:35:54,122 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2023-12-07 10:35:54,126 INFO sqlalchemy.engine.Engine [cached since 10.12s ago] ()
('sandy', 'Sandy Miles')


In [9]:
row = session.execute(select(User.name, Address.email_address).where(User.id == Address.user_id).order_by(Address.id)).fetchall()
print(row)

2023-12-07 10:36:03,178 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2023-12-07 10:36:03,183 INFO sqlalchemy.engine.Engine [generated in 0.00560s] ()
[('sandy', 'sandy@aol.com'), ('tony', 'tony.start@aol.com'), ('thor', 'baby.thor@aol.com')]


In [10]:
row = session.execute(select(("user details: " + User.name).label("username")).where(User.id == Address.user_id).order_by(Address.id)).fetchall()
print(row)

2023-12-07 10:36:17,475 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2023-12-07 10:36:17,478 INFO sqlalchemy.engine.Engine [generated in 0.00303s] ('user details: ',)
[('user details: sandy',), ('user details: tony',), ('user details: thor',)]


In [21]:

row = session.execute(select(literal_column("'user_details'").label("p"), User.name, Address.email_address).where(User.id == Address.user_id).order_by(Address.id)).fetchall()
for rows in row:
    print(f"{rows.p}: {rows.name}, {rows.email_address}")


2023-12-07 10:46:01,840 INFO sqlalchemy.engine.Engine SELECT 'user_details' AS p, user_account.name, address.email_address 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2023-12-07 10:46:01,844 INFO sqlalchemy.engine.Engine [cached since 526.6s ago] ()
user_details: sandy, sandy@aol.com
user_details: tony, tony.start@aol.com
user_details: thor, baby.thor@aol.com


In [40]:
row = select(User.name, Address.email_address)
print(row)

SELECT user_account.name, address.email_address 
FROM user_account, address


In [21]:
print(User.id == 1)
row = select(User).where(and_(or_(User.name == "sandy", User.name == "tony"), User.id == 1))

user_account.id = :id_1


In [58]:
row = select(User.name, Address.email_address).join(Address)
print(row)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [46]:
row = select(User, Address.email_address).join(Address)
print(row)

SELECT user_account.id, user_account.name, user_account.fullname, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [61]:
row = select(Address.email_address).select_from(User).join(Address)
print(row)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [65]:
row = select(func.count("*")).select_from(User)
print(row)

SELECT count(:count_2) AS count_1 
FROM user_account


In [67]:
#setting JOIN Clause
row = select(Address.email_address).select_from(User).join(Address, User.id == Address.user_id)
print(row)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [73]:
# left outer join
row = select(User, Address.email_address).join(Address, isouter=True)
print(row)

SELECT user_account.id, user_account.name, user_account.fullname, address.email_address 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id


In [4]:
# order by
row = select(User).order_by(User.name.desc())
print(row)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name DESC


In [9]:
# Aggregate functions with GROUP BY / HAVING

count_func = func.count(Address.id)
print(count_func)

row = select(User.id, func.count(Address.id).label("count")).join(Address).group_by(User.name).having(func.count(Address.id) > 0)
print(row)

count(address.id)
SELECT user_account.id, count(address.id) AS count 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > :count_1


In [4]:
row = select(Address.user_id, func.count(Address.id).label("num_addresses")).group_by("user_id").order_by("user_id", desc("num_addresses"))
print(row)

SELECT address.user_id, count(address.id) AS num_addresses 
FROM address GROUP BY address.user_id ORDER BY address.user_id, num_addresses DESC


In [7]:
user_alias_1 = aliased(User)
user_alias_2 = aliased(User)
row = select(user_alias_1.name, user_alias_2.name).join_from(user_alias_1, user_alias_2, user_alias_1.id > user_alias_2.id)
print(row)

SELECT user_account_1.name, user_account_2.name AS name_1 
FROM user_account AS user_account_1 JOIN user_account AS user_account_2 ON user_account_1.id > user_account_2.id


In [3]:
#Subquery for orm.entities

subq = select(Address).where(~Address.email_address.like("%@aol.com")).subquery()
address_subq = aliased(Address, subq)
print(address_subq)
row = select( User, address_subq).join_from(User, address_subq).order_by(User.id, address_subq.id)
print(row)

aliased(Address)
SELECT user_account.id, user_account.name, user_account.fullname, anon_1.id AS id_1, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN (SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE :email_address_1) AS anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id


In [5]:
#Common table expressions
subq = select(Address).where(~Address.email_address.like("%@example.com")).cte()
print(subq)
address_subq = aliased(Address, subq)
row = select(User.name, address_subq).join_from(User, address_subq).order_by(User.id, address_subq.id)
print(row)


SELECT address.id, address.email_address, address.user_id 
FROM address 
WHERE address.email_address NOT LIKE :email_address_1
WITH anon_1 AS 
(SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE :email_address_1)
 SELECT user_account.name, anon_1.id, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id


In [6]:
with Session(engine) as session:
    print(row)
    result = session.execute(row).all()
    print(result)

WITH anon_1 AS 
(SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE :email_address_1)
 SELECT user_account.name, anon_1.id, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id
2023-12-09 10:33:54,297 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-09 10:33:54,320 INFO sqlalchemy.engine.Engine WITH anon_1 AS 
(SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE ?)
 SELECT user_account.name, anon_1.id, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id
2023-12-09 10:33:54,327 INFO sqlalchemy.engine.Engine [generated in 0.00770s] ('%@example.com',)


[('sandy', Address(id=1, user_id=1, email_address=sandy@aol.com)), ('tony', Address(id=2, user_id=2, email_address=tony.start@aol.com)), ('thor', Address(id=3, user_id=3, email_address=baby.thor@aol.com))]
2023-12-09 10:33:54,338 INFO sqlalchemy.engine.Engine ROLLBACK
